In [1]:
import transformers
import datasets
import pprint
import html
import pandas
import IPython, IPython.display
import torch

# §5.1 创建数据集

In [2]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz -P ./dataset/
!7z x ./dataset/SQuAD_it-train.json.gz -o./dataset/

!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz -P ./dataset
!7z x ./dataset/SQuAD_it-test.json.gz -o./dataset/

[0] Downloading 'https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz' ...
HTTP response 302  [https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz]
Adding URL: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
[0] Downloading 'https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz' ...
Saving './dataset//SQuAD_it-train.json.gz'
HTTP response 200  [https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz]


No CAs were found in 'C:\ProgramData/ssl/ca-bundle.pem'



7-Zip 22.01 (x64) : Copyright (c) 1999-2022 Igor Pavlov : 2022-07-15

Scanning the drive for archives:
1 file, 7725286 bytes (7545 KiB)

Extracting archive: .\dataset\SQuAD_it-train.json.gz
--
Path = .\dataset\SQuAD_it-train.json.gz
Type = gzip
Headers Size = 30

Everything is Ok

Size:       43605829
Compressed: 7725286
[0] Downloading 'https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz' ...
HTTP response 302  [https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz]
Adding URL: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-test.json.gz
[0] Downloading 'https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-test.json.gz' ...
Saving './dataset/SQuAD_it-test.json.gz.1'
HTTP response 200  [https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-test.json.gz]


No CAs were found in 'C:\ProgramData/ssl/ca-bundle.pem'



7-Zip 22.01 (x64) : Copyright (c) 1999-2022 Igor Pavlov : 2022-07-15

Scanning the drive for archives:
1 file, 1051245 bytes (1027 KiB)

Extracting archive: .\dataset\SQuAD_it-test.json.gz
--
Path = .\dataset\SQuAD_it-test.json.gz
Type = gzip
Headers Size = 29

Everything is Ok

Size:       8385528
Compressed: 1051245


In [3]:
# 加载单文件

raw_datasets = datasets.load_dataset(
    "json",
    data_files="./dataset/SQuAD_it-train.json",
    field="data"
)
# 原始JSON文件需要获取其data属性，所以要指定load_dataset(field="data")
# {
#     "data": [
#         {'context'}, {}, {}, ...
#     ]
# }
pprint.pprint({
    "raw_datasets": raw_datasets,
})

{'raw_datasets': DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})}


In [4]:
# 加载多文件

raw_datasets = datasets.load_dataset(
    "json",
    data_files = {
        "train": "./dataset/SQuAD_it-train.json",
        "test": "./dataset/SQuAD_it-test.json"
    },
    field="data"
)
pprint.pprint({
    "raw_datasets": raw_datasets,
})

{'raw_datasets': DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})}


# §5.2 数据集切片

In [5]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip" -P ./dataset/
!7z x ./dataset/drugsCom_raw.zip -o./dataset/

[0] Downloading 'https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip' ...
Saving './dataset//drugsCom_raw.zip'
HTTP response 200 OK [https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip]


No CAs were found in 'C:\ProgramData/ssl/ca-bundle.pem'



7-Zip 22.01 (x64) : Copyright (c) 1999-2022 Igor Pavlov : 2022-07-15

Scanning the drive for archives:
1 file, 42989872 bytes (41 MiB)

Extracting archive: .\dataset\drugsCom_raw.zip
--
Path = .\dataset\drugsCom_raw.zip
Type = zip
Physical Size = 42989872

Everything is Ok

Files: 2
Size:       112360341
Compressed: 42989872


In [2]:
# 使用slice()方法进行切片

raw_datasets = datasets.load_dataset(
    "csv",
    data_files={
        "train": "./dataset/drugsComTrain_raw.tsv",
        "test": "./dataset/drugsComTest_raw.tsv"
    },
    delimiter="\t"
)
pprint.pprint({
    "raw_datasets": raw_datasets,
    "raw_datasets_train": raw_datasets["train"],
    "raw_datasets_train_slice": raw_datasets["train"].shuffle(seed=42).select(range(1000))
})

{'raw_datasets': DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
}),
 'raw_datasets_train': Dataset({
    features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
    num_rows: 161297
}),
 'raw_datasets_train_slice': Dataset({
    features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
    num_rows: 1000
})}


In [3]:
# 编辑数据集的列

# rename_column()重命名列
raw_datasets.rename_column(
    original_column_name="Unnamed: 0",
    new_column_name="patient_id"
)

# filter()删除空行，map()编辑列批量转小写
raw_datasets = raw_datasets.filter(lambda x: x["condition"] is not None)
raw_datasets.map(lambda x: {"condition": x["condition"].lower()})

# map()创建列
raw_datasets = raw_datasets.map(lambda x: {"review_length": len(x["review"].split())})

# sort()排序行
pprint.pprint(
    raw_datasets["train"].sort("review_length")[:3]
)

# map(batched=True)批处理加速运算
raw_datasets = raw_datasets.map(
    lambda x: {"review": html.unescape(x["review"])},
    batched=True
)

# map(batched=True, num_proc=8)多进程批处理加速运算
raw_datasets = raw_datasets.map(
    lambda x: {"review": html.unescape(x["review"])},
    batched=True
)


{'Unnamed: 0': [111469, 13653, 53602],
 'condition': ['Hepatitis C', 'ADHD', 'Birth Control'],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'drugName': ['Ledipasvir / sofosbuvir',
              'Amphetamine / dextroamphetamine',
              'Alesse'],
 'rating': [10.0, 10.0, 10.0],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'review_length': [1, 1, 1],
 'usefulCount': [41, 3, 0]}


In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

raw_datasets.map(
    lambda x: tokenizer(x["review"], truncation=True),
    batched=True
)

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160398
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 53471
    })
})

In [ ]:
# tokenized_datasets = raw_datasets.map(
#     lambda x: tokenizer(
#         x["review"], 
#         truncation=True, 
#         max_length=128, 

#         return_overflowing_tokens=True
#         # 这会导致返回的raw_datasets["train"][0]["input_ids"]不是list[int]，而是list[list[int]]
#         # map()默认的batch_size为1000，但是却接收了多余1000个的list[int]，导致错误
#     ), 
#     batched=True
# )

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

ArrowInvalid: Column 8 named input_ids expected length 1000 but got length 1442

In [6]:
# 解决方法一：删除旧列中的字段，就不用担心1000!=1442了
tokenized_datasets = raw_datasets.map(
    lambda x: tokenizer(
        x["review"], 
        truncation=True, 
        max_length=128, 
        return_overflowing_tokens=True
    ), 
    batched=True,
    remove_columns=raw_datasets["train"].column_names # 删除旧列
)

raw_datasets, tokenized_datasets

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
         num_rows: 160398
     })
     test: Dataset({
         features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
         num_rows: 53471
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
         num_rows: 234877
     })
     test: Dataset({
         features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
         num_rows: 78315
     })
 }))

In [9]:
# 解决方法二：使新列和旧列的行数相等

def tokenize_and_split(x):
    result = tokenizer(
        x["review"], 
        truncation=True, 
        max_length=128, 
        return_overflowing_tokens=True,
    )
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in x.items():
        result[key] = [values[i] for i in sample_map]
    return result

tokenized_datasets = raw_datasets.map(
    tokenize_and_split,
    batched=True,
)

raw_datasets, tokenized_datasets

(DatasetDict({
     train: Dataset({
         features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
         num_rows: 160398
     })
     test: Dataset({
         features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
         num_rows: 53471
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 234877
     })
     test: Dataset({
         features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 78315
     })
 }))

In [10]:
# 将raw_datasets中的训练集的20%作为验证集，测试集保持不变

raw_datasets_temp = raw_datasets["train"].train_test_split(train_size=0.8, seed=42)
raw_datasets_temp["validation"] = raw_datasets_temp.pop("test")
# raw_datasets_temp: {"train": ..., "test": ...}

raw_datasets["train"] = raw_datasets_temp["train"]
raw_datasets["validation"] = raw_datasets_temp["validation"]
# raw_datasets: {"train": ..., "validation": ..., "test": ...}

pprint.pprint(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 128318
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 53471
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 32080
    })
})


In [ ]:
# 保存为Apache Arrow
# raw_datasets.save_to_disk("path")           

# 保存为JSON
# for split, dataset in raw_datasets.items():
#     raw_datasets.to_csv("filename-{split}.json")

# 保存为CSV
# for split, dataset in raw_datasets.items():
#     raw_datasets.to_json("filename-{split}.csv")

# 加载Apache Arrow
# raw_datasets = datasets.load_from_disk("path")   

# 加载JSON
# raw_datasets = datasets.load_dataset("json", data_files={"test": "...", "validation": "...", "test": "..."})

# 加载CSV
# raw_datasets = datasets.load_dataset("csv", data_files={"test": "...", "validation": "...", "test": "..."})

# §5.3 大规模数据集内存映射

In [3]:
data_files = ""
pubmed_dataset = datasets.load_dataset(
    "json", 
    data_files="https://the-eye.eu/public/AI/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst", 
    split="train"
)

pprint.pprint(pubmed_dataset)

# 原数据集下载链接失效，此处略。参见https://huggingface.co/learn/nlp-course/en/chapter5/4?fw=pt#big-data-datasets-to-the-rescue

FileNotFoundError: Unable to find 'https://the-eye.eu/public/AI/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst'

# §5.4 上传数据集
略，详见https://huggingface.co/learn/nlp-course/en/chapter5/5?fw=pt

In [4]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

pprint.pprint(response.json())

[{'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'NONE',
  'body': '### Feature request\n'
          '\n'
          'There should be a way to specify the datatype of a column in '
          '`datasets.add_column()`. \n'
          '\n'
          '### Motivation\n'
          '\n'
          'To specify a custom datatype, we have to use '
          '`datasets.add_column()` followed by `datasets.cast_column()` which '
          'is slow for large datasets. Another workaround is to pass a '
          '`numpy.array()` of desired type to the `datasets.add_column()` '
          'function.\r\n'
          '\r\n'
          'IMO this functionality should be natively supported.\r\n'
          '\r\n'
          'https://discuss.huggingface.co/t/add-column-with-a-particular-type-in-datasets/95674\r\n'
          '\r\n'
          '\n'
          '\n'
          '### Your contribution\n'
          '\n'
          'I can submit a PR for this.',
  'closed_at': None,


# §5.5 FAISS文本语义向量检索

In [2]:
# 下载文本数据集

issues_dataset = datasets.load_dataset(
    "lewtun/github-issues",
    split="train"
)

pprint.pprint(issues_dataset)

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})


In [3]:
# 清洗数据集

issues_dataset = issues_dataset.filter(
    lambda x: x["is_pull_request"] == False
).filter(
    lambda x: len(x["comments"]) > 0
).remove_columns(
    set(["title", "body", "html_url", "comments"]).symmetric_difference(issues_dataset.column_names)
)

pprint.pprint(issues_dataset)

Filter:   0%|          | 0/3019 [00:00<?, ? examples/s]

Filter:   0%|          | 0/984 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})


In [4]:
issues_dataset.set_format("pandas")

dataframe: pandas.DataFrame = issues_dataset[:]
IPython.display.display(IPython.display.HTML(dataframe.head(3).to_html()))

dataframe = dataframe.explode("comments", ignore_index=True)
# Dataframe.explode(key: str)：将dataframe[key]: list[any]进行笛卡尔积级别的复制，复制为若干dataframe[key]: any，类似于数据库中的范式下调
IPython.display.display(IPython.display.HTML(dataframe.head(3).to_html()))

In [5]:
comments_dataset = datasets.Dataset.from_pandas(dataframe)
pprint.pprint(comments_dataset)

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})


In [6]:
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())},
).filter(
    lambda x: x["comment_length"] > 15
)

pprint.pprint(comments_dataset)

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})


In [7]:
comments_dataset = comments_dataset.map(
    lambda x: {
        "text": x["title"] + " \n " + x["body"] + " \n " + x["comments"]
    }
)
pprint.pprint(comments_dataset)

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text'],
    num_rows: 2175
})


In [8]:
# 创建向量化模型

model = transformers.AutoModel.from_pretrained(
    "sentence-transformers/multi-qa-mpnet-base-dot-v1"
).to(torch.device("cuda"))
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "sentence-transformers/multi-qa-mpnet-base-dot-v1"
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

c:\Users\Yaner\Desktop\Thoughts\.conda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# 生成Embedding

def get_embeddings(text_list: str | list[str]):
    encoded_input = tokenizer(
        text_list, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )
    encoded_input = {k: v.to(torch.device("cuda")) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return model_output.last_hidden_state[:, 0]

embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

pprint.pprint(embeddings_dataset)

embeddings_dataset.add_faiss_index(column="embeddings")

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [11]:
# 查询Embedding

question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)
scores, samples

(array([22.406654, 22.894001, 24.148983, 24.555511, 25.505013],
       dtype=float32),
 {'html_url': ['https://github.com/huggingface/datasets/issues/824',
   'https://github.com/huggingface/datasets/issues/824',
   'https://github.com/huggingface/datasets/issues/824',
   'https://github.com/huggingface/datasets/issues/824',
   'https://github.com/huggingface/datasets/issues/824'],
  'title': ['Discussion using datasets in offline mode',
   'Discussion using datasets in offline mode',
   'Discussion using datasets in offline mode',
   'Discussion using datasets in offline mode',
   'Discussion using datasets in offline mode'],
  'comments': ['here is my way to load a dataset offline, but it **requires** an online machine\r\n1. (online machine)\r\n```\r\nimport datasets\r\ndata = datasets.load_dataset(...)\r\ndata.save_to_disk(/YOUR/DATASET/DIR)\r\n```\r\n2. copy the dir from online to the offline machine\r\n3. (offline machine)\r\n```\r\nimport datasets\r\ndata = datasets.load_from_dis